In [1]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

/home/mserrao/miniconda3/envs/magi/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-base')
model = AutoModel.from_pretrained('facebook/dinov2-base')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs[0]

In [6]:
display(outputs)
display(last_hidden_states.shape)

BaseModelOutputWithPooling(last_hidden_state=tensor([[[-2.2005, -0.4495,  1.0964,  ...,  0.2113,  1.1496,  0.1267],
         [-3.3959, -0.8942, -1.0315,  ...,  1.4867, -0.5316, -0.9317],
         [-2.9354,  1.1564, -0.7656,  ...,  0.6501, -1.9041, -2.3671],
         ...,
         [-0.4961,  1.3416, -0.2869,  ...,  0.2186, -2.9740,  0.5224],
         [-3.0983,  2.0506,  1.4553,  ...,  0.7989, -2.6633,  2.2196],
         [-0.6799,  0.0304,  1.4230,  ..., -0.3459, -2.4010, -0.1477]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-2.2005e+00, -4.4949e-01,  1.0964e+00, -1.3291e+00, -2.9671e-02,
          7.5587e-01, -1.5445e-02,  1.5769e+00, -1.9610e+00,  2.4383e+00,
         -4.3804e-01, -1.3045e+00,  1.1043e+00,  1.4173e+00, -9.2579e-01,
         -1.6590e+00,  2.3838e+00,  5.3869e-02, -8.6847e-02,  1.9271e+00,
          3.9997e-01, -1.2583e+00, -2.9034e+00,  2.3388e+00,  5.0291e-01,
         -1.3369e+00, -7.6849e-01,  4.9257e-01, -1.5086e+00,  5.1088e-01,
         -1

torch.Size([1, 257, 768])

In [4]:
# We have to force return_dict=False for tracing
model.config.return_dict = False

with torch.no_grad():
    traced_model = torch.jit.trace(model, [inputs.pixel_values])
    traced_outputs = traced_model(inputs.pixel_values)

print((last_hidden_states - traced_outputs[0]).abs().max())

/home/mserrao/miniconda3/envs/magi/lib/python3.8/site-packages/transformers/modeling_utils.py:4779: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
/home/mserrao/miniconda3/envs/magi/lib/python3.8/site-packages/transformers/models/dinov2/modeling_dinov2.py:164: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if num_channels != self.num_channels:


tensor(0., grad_fn=<MaxBackward1>)
